# Campervan Projekt

## Libraries and settings

In [1]:
# Libraries
import os
import pprint
import requests
import numpy as np
import pandas as pd

import geopandas as gpd

from shapely.geometry import Polygon

import matplotlib.patches as patches
from matplotlib import pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

/workspaces/project_Autoscout_Campers/Project_Campers


## Import data

In [3]:
## Read data
df = pd.read_csv("./data/Autoscout24_Camper.csv", sep=",")
df


,web-scraper-order,web-scraper-start-url,autoscout24,autoscout24-href,description_raw,price_raw,mileage_raw,fuel_type_raw,location_raw
0,1733470796-1,https://www.autoscout24.ch/de/s/vc-camper?mile...,NaN,https://www.autoscout24.ch/de/d/mercedes-benz-...,MERCEDES-BENZ Camper,CHF 15'900.–,265'000 km,Diesel,"Mittelstrasse 38, 3613 Steffisburg"
1,1733470799-2,https://www.autoscout24.ch/de/s/vc-camper?mile...,NaN,https://www.autoscout24.ch/de/d/carado-i-449-e...,CARADO I 449 Édition 15 Fiat Capron 169L/1449,CHF 71'900.–,14'000 km,Diesel,"Zone industrielle Les Portettes 2c, 1312 Eclépens"
2,1733470801-3,https://www.autoscout24.ch/de/s/vc-camper?mile...,NaN,https://www.autoscout24.ch/de/d/dethleffs-fiat...,DETHLEFFS Fiat Ducato,CHF 37'800.–,90'000 km,Diesel,"Rte du St-Bernard 8, 1937 Orsières"
3,1733470805-4,https://www.autoscout24.ch/de/s/vc-camper?mile...,NaN,https://www.autoscout24.ch/de/d/fiat-euro-mast...,FIAT Euro Master Wohnmobil,CHF 19'900.–,187'236 km,Diesel,"Badenerstrasse 84, 8952 Schlieren"
4,1733470807-5,https://www.autoscout24.ch/de/s/vc-camper?mile...,NaN,https://www.autoscout24.ch/de/d/vw-t61-califor...,VW T6.1 California 2.0 TDI Ocean Edition Liber...,CHF 84'500.–,16'500 km,Diesel,8852 Altendorf
...,...,...,...,...,...,...,...,...,...
1582,1733477100-1583,https://www.autoscout24.ch/de/s/vc-camper?mile...,NaN,https://www.autoscout24.ch/de/d/sunlight-cliff...,"SUNLIGHT Cliff 600 Adventure Edition, Citroen ...",CHF 52'900.–,11'000 km,Diesel,8406 Winterthur
1583,1733477103-1584,https://www.autoscout24.ch/de/s/vc-camper?mile...,NaN,https://www.autoscout24.ch/de/d/laika-kosmo-v9...,"LAIKA Kosmo V9 Compact, Fiat Ducato 2.3MJ 140PS",CHF 56'900.–,14'500 km,Diesel,8406 Winterthur
1584,1733477106-1585,https://www.autoscout24.ch/de/s/vc-camper?mile...,NaN,https://www.autoscout24.ch/de/d/peugeot-335-12...,PEUGEOT 335-120 FL2,CHF 32'777.–,132'000 km,Diesel,"Sulzer-Allee 41, 8404 Winterthur"
1585,1733477109-1586,https://www.autoscout24.ch/de/s/vc-camper?mile...,NaN,https://www.autoscout24.ch/de/d/citroen-22-hdi...,CITROEN 2.2 HDI 12Q,CHF 31'777.–,116'700 km,Diesel,"Sulzer-Allee 41, 8404 Winterthur"
